In [1]:
import pandas as pd
import numpy as np


#  Olympic data set analysis

Often people wonder if how a country performs is related to its, Economic status. it isbelived that people from more economically adept places have a better chance at plaacing at the games. people also wonder if hosting an olypmic games provides a siginficant boost to the economy that they are in. This anlysis will inclue the ETL, of various relevant documents such as medal data, olympic perfromance and partipatyion data, into an organisaed warehouse where visualisations will stem from to answer the questions above.

0lypmic performance will be measures in medal counts of that partcular country, and economic perforamce indicated by the gross domestic product of that partcular country (GDP), the data used will stem from 2000 olympics to the current 2023 olympic games to provide a recent but comprhensive suite of data.

## ETL Process 
Below we conduct the etl, process creating our different dimensions and then fact tables. once we do that we will then connect to a pgadmin remost server and from there build hierachies and so forth.

## Extraction

Since the data was just simplem, csv downlaod the extraction process will just be a series of code where we just load the data into the our staging area the jupyter notebook.


In [2]:
athlete_medal_data_raw = "AthleteMedalData.csv"
gdp_data_raw = "gdp data"

In [ ]:
# def open_medal_athelte_data(data):
#     """ This function will just open the original athelte
#         and medal raw csv into our staging area, and then 
#         it will be converted into a python pandas dataframe """
    
#     datalist = []
#     with open(data, "rt") as raw_ath_med:
#         ath_med = raw_ath_med.read()
#         lines  = ath_med.splitlines()

#         for line in lines[1:]:
#             split_lines = line.split(',')

#             player_id = split_lines[0]
#             Name = split_lines[1]
#             Sex = split_lines[2]
#             Team = split_lines[3]
#             NOC = split_lines[4]
#             Year = split_lines[5]
#             Season = split_lines[6]
#             City = split_lines[7]
#             Sport = split_lines[8]
#             Event = split_lines[9]
#             Medal = split_lines[10]

#             datalist.append((player_id,Name,Sex,Team,NOC,Year,Season,City,Sport,Event,Medal))

#     return datalist 

# medal_athelte_data = open_medal_athelte_data(athlete_medal_data_raw)


In [ ]:
import csv

def open_medal_athelte_data(data):
    """ This function will open the original athlete
        and medal raw CSV into our staging area, and then
        convert it into a Python pandas dataframe. """
    
    datalist = []
    with open(data, "rt") as raw_ath_med:
        reader = csv.reader(raw_ath_med)
        next(reader)  # Skip the header row
        
        for row in reader:
            # Ensuring we have the right number of columns
            if len(row) == 11:
                player_id, Name, Sex, Team, NOC, Year, Season, City, Sport, Event, Medal = row
                datalist.append((player_id, Name, Sex, Team, NOC, Year, Season, City, Sport, Event, Medal))
    
    return datalist

# Example usage
medal_athelte_data = open_medal_athelte_data(athlete_medal_data_raw)

In [17]:
medal_athelte_data_df = pd.DataFrame(medal_athelte_data, columns = ["player_id", "Name", "Sex", "Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"])
medal_athelte_data_df

,player_id,Name,Sex,Team,NOC,Year,Season,City,Sport,Event,Medal
0,0,A Dijiang,M,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,1,A Lamusi,M,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No medal
2,2,Gunnar Aaby,M,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,No medal
3,3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,26,Cornelia (-strannood),F,Netherlands,NED,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,No medal
...,...,...,...,...,...,...,...,...,...,...,...
252560,4986655,Sefora Ada,F,Equatorial Guinea,GEQ,2024,Summer,Paris,Athletics,Women's 100m,No medal
252561,9460001,Emanuela Liuzzi,F,Italy,ITA,2024,Summer,Paris,Wrestling,Women's Freestyle 50kg,No medal
252562,1972077,Isayah Boers,M,Netherlands,NED,2024,Summer,Paris,Athletics,4 x 400m Relay Mixed,Gold
252563,1899865,Kevin Staut,M,France,FRA,2024,Summer,Paris,Equestrian,Jumping Team,Bronze


Now that we have loaded in the data, let us investigate if there is any missig entries for anyof the fields.

In [18]:
medal_athelte_data_df.describe()

,player_id,Name,Sex,Team,NOC,Year,Season,City,Sport,Event,Medal
count,252565,252565,252565,252565,252565,252565,252565,252565,252565,252565,252565
unique,235903,129992,2,1193,234,31,1,23,76,1041,4
top,1551050,John Jr.,M,United States,USA,2020,Summer,Tokyo,Athletics,Football Men's Football,No medal
freq,9,76,178544,16236,16774,15121,252565,22823,43294,5733,213747


In [19]:
medal_athelte_data_df.isna().sum()

player_id    0
Name         0
Sex          0
Team         0
NOC          0
Year         0
Season       0
City         0
Sport        0
Event        0
Medal        0
dtype: int64

For every element there is no missing entries.
We will move unto remove those columns where the, Medal type is "No medal" as our analysis will not be focused on collective particpation rates rather on the performance of those countries that particpated. 

In [20]:
medal_athelte_data_df = medal_athelte_data_df[medal_athelte_data_df["Medal"] != "No medal"]

In [28]:

pd.set_option('display.max_rows', 350)  # Show all rows
pd.set_option('display.max_columns', 350)

medal_athelte_data_df.head(20)

,player_id,Name,Sex,Team,NOC,Year,Season,City,Sport,Event,Medal
3,3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
12,37,Arvo Aaltonen,M,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
13,38,Arvo Aaltonen,M,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
15,41,Paavo Aaltonen,M,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze
16,42,Paavo Aaltonen,M,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Team All-Around,Gold
18,44,Paavo Aaltonen,M,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Horse Vault,Gold
22,48,Paavo Aaltonen,M,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Pommelled Horse,Gold
24,50,Paavo Aaltonen,M,Finland,FIN,1952,Summer,Helsinki,Gymnastics,Gymnastics Men's Team All-Around,Bronze
33,79,Ragnhild Aamodt,F,Norway,NOR,2008,Summer,Beijing,Handball,Handball Women's Handball,Gold
37,86,Alf Aanning,M,Norway,NOR,1920,Summer,Antwerpen,Gymnastics,"Gymnastics Men's Team All-Around, Free System",Silver


In [49]:
year_df  = medal_athelte_data_df[["Year","City"]]
year_df.drop_duplicates(subset=["Year", "City"], keep="first", inplace=True)

# adding a unique identifier to the newly made dataframe
year_df = year_df.reset_index()
year_df["Year_id"] = year_df.index
year_df = year_df.drop(columns=["index"])
year_df = year_df[["Year_id","Year","City"]]
year_df 

C:\Users\Maria\AppData\Local\Temp\ipykernel_25008\2341456113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df.drop_duplicates(subset=["Year", "City"], keep="first", inplace=True)


,Year_id,Year,City
0,0,1900,Paris
1,1,1920,Antwerpen
2,2,1948,London
3,3,1952,Helsinki
4,4,2008,Beijing
5,5,1988,Seoul
6,6,1996,Atlanta
7,7,1912,Stockholm
8,8,1956,Melbourne
9,9,2016,Rio de Janeiro


In [50]:
year_df.to_csv('year_dim.csv', index=False) 

The next part is to tranform the athelte medal data into an event dataframe

In [53]:
event_df = medal_athelte_data_df[["Sport","Event"]]
event_df.drop_duplicates(subset=["Sport","Event"], keep="first", inplace=True)
event_df = event_df.reset_index()
event_df["Event_id"] = event_df.index
event_df = event_df.drop(columns=["index"])
event_df = event_df[["Event_id", "Sport","Event"]]
event_df

C:\Users\Maria\AppData\Local\Temp\ipykernel_25008\2537558412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df.drop_duplicates(subset=["Sport","Event"], keep="first", inplace=True)


,Event_id,Sport,Event
0,0,Tug-Of-War,Tug-Of-War Men's Tug-Of-War
1,1,Swimming,Swimming Men's 200 metres Breaststroke
2,2,Swimming,Swimming Men's 400 metres Breaststroke
3,3,Gymnastics,Gymnastics Men's Individual All-Around
4,4,Gymnastics,Gymnastics Men's Team All-Around
...,...,...,...
1086,1086,Swimming,Mixed 4 x 100m Medley Relay
1087,1087,Swimming,Women's 4 x 100m Medley Relay
1088,1088,Swimming,Women's 4 x 200m Freestyle Relay
1089,1089,Water Polo,Women


Now let us create the event dimension csv.

In [56]:
event_df.to_csv("event_dim.csv", index= False)